# Project: JSON Based Data Exercise
****
1. Find the 10 countries with most projects.
2. Find the top 10 major project themes. 
3. Create a dataframe with missing theme names filled in.

In [172]:
#imports
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np


# Problem 1: find the countries with the most projects

In [173]:
json_string = json.load((open('data/world_bank_projects.json')))

In [174]:
df = json_normalize(json_string)

In [175]:
df.head()

,_id.$oid,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,source,status,supplementprojectflg,theme1.Name,theme1.Percent,theme_namecode,themecode,totalamt,totalcommamt,url
0,52b213b38594d8a2be17c780,1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,IBRD,Active,N,Education for all,100,"[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,52b213b38594d8a2be17c781,2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,IBRD,Active,N,Other economic management,30,"[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,52b213b38594d8a2be17c782,2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,IBRD,Active,Y,Regional integration,46,"[{u'code': u'47', u'name': u'Regional integrat...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,52b213b38594d8a2be17c783,2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,IBRD,Active,N,Participation and civic engagement,50,"[{u'code': u'57', u'name': u'Participation and...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,52b213b38594d8a2be17c784,2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,IBRD,Active,N,Export development and competitiveness,30,"[{u'code': u'45', u'name': u'Export developmen...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [176]:
#check for duplicates
df.id.value_counts(dropna=False).head()

P120867    1
P127482    1
P112611    1
P146021    1
P129742    1
Name: id, dtype: int64

### 1. Countries with the most projects

In [177]:
#find 10 countries with the most projects
top_countries = df.countryname.value_counts().head(10)
print(top_countries)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64


# Problems 2 & 3: find the top 10 major project themes & create a dataframe with the missing names filled in. 

In [178]:
#normalize mj themes 
mjthemes_df = json_normalize(json_string, 'mjtheme_namecode',['countryname', 'countrycode','id'], errors = 'ignore' )

In [179]:
mjthemes_df.head(10)

,code,name,countryname,id,countrycode
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828,ET
1,11,,Federal Democratic Republic of Ethiopia,P129828,ET
2,1,Economic management,Republic of Tunisia,P144674,TN
3,6,Social protection and risk management,Republic of Tunisia,P144674,TN
4,5,Trade and integration,Tuvalu,P145310,TV
5,2,Public sector governance,Tuvalu,P145310,TV
6,11,Environment and natural resources management,Tuvalu,P145310,TV
7,6,Social protection and risk management,Tuvalu,P145310,TV
8,7,Social dev/gender/inclusion,Republic of Yemen,P144665,RY
9,7,Social dev/gender/inclusion,Republic of Yemen,P144665,RY


In [180]:
#drop rows with no names and drop duplicates
themecode_df = themecode_df[themecode_df.name != ''].drop_duplicates()
themecode_df

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [185]:
#create a dictionary of codes and names
theme_dict = dict(zip(themecode_df.code, themecode_df.name))
#replace empty strings with NaN
mjthemes_df = mjthemes_df.replace('', np.nan)


### 3. Dataframe with completed name column

In [186]:
#create a dataframe with missing names filled in.
mjthemes_df.name = mjthemes_df.name.fillna(mjthemes_df.code.map(theme_dict))
mjthemes_df.head()

,code,name,countryname,id,countrycode
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828,ET
1,11,Environment and natural resources management,Federal Democratic Republic of Ethiopia,P129828,ET
2,1,Economic management,Republic of Tunisia,P144674,TN
3,6,Social protection and risk management,Republic of Tunisia,P144674,TN
4,5,Trade and integration,Tuvalu,P145310,TV


### 2. Top 10 major project themes

In [141]:
#find the top 10 major project themes
mjthemes_df.name.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64